In [1]:
import pandas as pd
import pandas
import time
import datetime as dt
from datetime import datetime, timedelta
from dateutil import relativedelta
import numpy as np
#pip install xlrd

In [2]:
# Input 
dropout='dropoutDiciembre.xlsx'
filename='HouseholdInstalled.xls'
#Start = input('Start Date (DD/MM/YYYY):')
#End=input('End Date (DD/MM/YYYY):')
Start='01/12/2022'
End='30/12/2022'

In [3]:
#BASE
df1 = pd.read_excel(io=dropout,keep_default_na=False,skiprows=9,usecols=lambda x: 'Unnamed' not in x,
                    parse_dates=False,converters={'Installation Date':lambda x: x.strftime("%d/%m/%Y") if type(x) == dt.datetime else x,'Disinstallation Date':lambda x: x.strftime("%d/%m/%Y") if type(x) == dt.datetime else x})
del df1['Region']
a=df1['City'].copy()
df1.insert(3,'Region',a)
df1=df1.drop(['Load Date'], axis=1, errors='ignore')
df1.rename(index={'Family Code': 'Family_Code'})
df1.index.drop_duplicates()
df1.index.dropna()  

#Formato
Start=pd.to_datetime(Start, format = "%d/%m/%Y")
End=pd.to_datetime(End, format = "%d/%m/%Y")
#Tercer Hoja
df3=pd.DataFrame()
df3['Ciudad']=['1 - Al-Riyadh','109 - Al-Baha','116 - Sakaka','21 - Makkah Almukarramah','22 - Jeddah',
 '23 - At-Taif','24 - Al-Qunfidhah','3 - Al-Kharj','33 - Al-Madinah','34 - Yanbu Al-Bahr',
 '4 - Ad-Duwadimi','40 - Buraydah','51 - Ad-Dammam','52 - Al-Ahsa','53 - Hafar Al-Batin',
 '54 - Al-Jubayl','56 - Al-Khubar','62 - Abha','63 - Khamis Mushayt','64 - Bishah','66 - Muhayil',
 '74 - Tabouk','80 - Hail','84 - Arar','6 - Aseer','10 - Makkah Al-Mokarramah','22 - Jeddah']
df3['Region']=['Central Region','Southern Region','Northern Region','Makkah Almukarramah','Jeddah','Western Region -Other Cities',
 'Western Region -Other Cities','Central Region','Madina','Western Region -Other Cities','Central Region','Northern Region',
 'Ad-Dammam','Eastern Region -Other Cities','Eastern Region -Other Cities','Eastern Region -Other Cities','Eastern Region -Other Cities',
 'Southern Region','Southern Region','Southern Region','Southern Region','Northern Region','Northern Region','Northern Region','Southern Region','Jeddah','Jeddah']

#Cleaning anything that's not 120,130,140,150,155,198 and 199
df1=df1[df1["Status"].str.contains('120|130|140|150|155|198|199')]

In [4]:
def replacement_status (df):
    column_98_to_99={'Status':{'198 -  Equipment to be collected':'199 -  Dropout'}}
    for columnName,dic_codes_to_status in [i for i in column_98_to_99.items() if i[0] in df.columns]:
        for exception in sorted(dic_codes_to_status.keys(),key=lambda x:(len(x),x),reverse=True):
            newstatus = dic_codes_to_status[exception]
            df[columnName]=df[columnName].astype(str).str.replace(exception,newstatus)
    return df

def replacement_sec_status (df):
    change_98_to_99={'Sec_Status':{'98K - Forced Turnover - QC No Compliance':'99K - Forced Turnover - QC No Compliance',
                                   '98T - Forced Turnover - Non-Cooperating home':'99T - Forced Turnover - Non-Cooperating home',
                                   '98C - Forced Turnover - PM Moving':'99C - Forced Turnover - PM Moving'}}
    for columnName,dic_codes_to_secstatus in [j for j in change_98_to_99.items() if j[0] in df.columns]:
        for exception2 in sorted(dic_codes_to_secstatus.keys(),key=lambda x:(len(x),x),reverse=True):
            newsecstatus = dic_codes_to_secstatus[exception2]
            df[columnName]=df[columnName].astype(str).str.replace(exception2,newsecstatus)
    return df


def replacement_codes_to_region (df):
    column_city_to_region={'Region':{'1 - Al-Riyadh':'Central Region','109 - Al-Baha':'Southern Region',
                                  '116 - Sakaka':'Northern Region','21 - Makkah Almukarramah':'Makkah Almukarramah',
                                  '22 - Jeddah':'Jeddah','23 - At-Taif':'Western Region -Other Cities','24 - Al-Qunfidhah':'Western Region -Other Cities',
                                  '3 - Al-Kharj':'Central Region','33 - Al-Madinah':'Madina','34 - Yanbu Al-Bahr':'Western Region -Other Cities',
                                  '4 - Ad-Duwadimi':'Central Region','40 - Buraydah':'Northern Region','51 - Ad-Dammam':'Ad-Dammam',
                                  '52 - Al-Ahsa':'Eastern Region -Other Cities','53 - Hafar Al-Batin':'Eastern Region -Other Cities', '54 - Al-Jubayl':'Eastern Region -Other Cities',
                                  '56 - Al-Khubar':'Eastern Region -Other Cities','62 - Abha':'Southern Region','63 - Khamis Mushayt':'Southern Region',
                                  '64 - Bishah':'Southern Region','66 - Muhayil':'Southern Region','74 - Tabouk':'Northern Region',
                                  '80 - Hail':'Northern Region','80 - Arar':'Northern Region','84 - Arar':'Northern Region'}}
    for columnName,dic_codes_to_region in [k for k in column_city_to_region.items() if k[0] in df.columns]:
        for city in sorted(dic_codes_to_region.keys(),key=lambda x:(len(x),x),reverse=True):
            region = dic_codes_to_region[city]
            df[columnName]=df[columnName].astype(str).str.replace(city,region)
    return df


def replacement_nationality (df):
    column_to_nationality={'Nationality':{'1 - Saudi':'Saudi','2 - Non-Saudi Arab':'Non-Saudi','3 - Non-Saudi Asian':'Non-Saudi',
                                          '4 - Non-Saudi Westerner':'Non-Saudi','5 - Non-Saudi African':'Non-Saudi'}}
    for column,dic_codes_to_nationality in [l for l in column_to_nationality.items() if l[0] in df.columns]:
        for nationality in sorted(dic_codes_to_nationality.keys(),key=lambda x:(len(x),x),reverse=True):
            newname = dic_codes_to_nationality[nationality]
            df[column]=df[column].astype(str).str.replace(nationality,newname)
    return df


def replacements (df):
    replacements=[replacement_status,replacement_sec_status,replacement_codes_to_region,replacement_nationality ]
    for i in replacements:
        df=(i)(df)
    return df

In [5]:
cols = ['Family_Code','Status','Sec_Status','Region','City','Technician','Installation_Date','Desinstallation_Date','Dwelling_Type',
        'Panel_Type','Nationality','Nr. of Active Individuals','Nr. Monitored Tv']
df1=df1.set_axis(cols, axis=1,copy=False)
df1 = df1[df1['Panel_Type']!= 1]
replacements(df1)
df1['Installation_Date']=pd.to_datetime(df1['Installation_Date'], format = "%d/%m/%Y",errors='coerce')
df1['Desinstallation_Date'] = pd.to_datetime(df1['Desinstallation_Date'], format = "%d/%m/%Y", errors='coerce')
Description=['Total','Total','Total','Total','Total','Total','Total','Total','Total','Total','Total','Saudi','Saudi','Saudi','Saudi','Non-Saudi','Non-Saudi','Non-Saudi','Total','Total','Total','Total','Total']
Info=['Saudi','Non-Saudi','Central Region','Ad-Dammam','Eastern Region -Other Cities','Northern Region','Southern Region','Jeddah','Madina','Makkah Almukarramah','Western Region -Other Cities',
'1-2 member households','3-4 member households','5-6 member households','7+ member households','1-2 member households','3-4 member households','5+ member households','Traditional house',
'Villa','Apartment','1 TV set','2+ TV sets']
Categories=['Nationality_Total_Saudi','Nationality_Total_Non-Saudi','Region and big Cities_Total_Central Region',
'Region and big Cities_Total_Ad-Dammam','Region and big Cities_Total_Eastern Region -Other Cities',
'Region and big Cities_Total_Northern Region','Region and big Cities_Total_Southern Region',
'Region and big Cities_Total_Jeddah','Region and big Cities_Total_Madina','Region and big Cities_Total_Makkah Almukarramah',
'Region and big Cities_Total_Western Region -Other Cities','Household size_Saudi_1-2 member households','Household size_Saudi_3-4 member households',
'Household size_Saudi_5-6 member households','Household size_Saudi_7+ member households','Household size_Non-Saudi_1-2 member households','Household size_Non-Saudi_3-4 member households',
'Household size_Non-Saudi_5+ member households','Type of Dwelling_Total_Traditional house','Type of Dwelling_Total_Villa','Type of Dwelling_Total_Apartment','TV set number_Total_1 TV set','TV set number_Total_2+ TV sets']

#Report
df=df1['Sec_Status'].drop_duplicates()
list_var_columns=df.to_list()
df2 = pd.DataFrame(df,columns=list_var_columns)
list_var_columns_99=[i for i in list_var_columns if i.startswith("99")]
df2=df2[list_var_columns_99]



NuevasColumnas=['Info','Description','Category','Total_Inst_Start_Date','Dropout_Total']
Content=[Info,Description,Categories,'','']
for i in range(len(NuevasColumnas)):
    df2.insert(i,NuevasColumnas[i],Content[i])


## Validation

In [6]:
df5 = pd.read_excel(io=filename,usecols=lambda x: 'Unnamed: 2' not in x)
df6=df1.copy()
cols = ['Household_Code','Installation_Date','Panel_Age_M','Area','Status_Code','Status_Description','Meter_Type','Total_Individuals','Monitored_TV',
        'Last_Tech_Visit','%Days_Intab']
df5=df5.set_axis(cols, axis=1,copy=False)

df5['Household_Code']=df5.Household_Code.drop_duplicates()

df6=df6[df6["Status"].str.contains('120|130|140|150|155')]
df6=df6.reset_index()
df6=df6.rename(columns={'Family Code': 'Family_Code'})
df6['Family_Code']=df6.Family_Code.drop_duplicates()
df6 = df6[pd.notnull(df6['Family_Code'])]

In [7]:
set_my_code=set(df6['Family_Code'].astype(int))
set_pollux_code=set(df5['Household_Code'])
set_code_faltantes=set_pollux_code-set_my_code
set_code_extras=set_my_code-set_pollux_code
df1=df1.drop(set_code_extras,axis=0)
df2=df2.drop(set_code_extras,axis=0)
df7=df6.loc[set_code_extras]
df1=df1.drop(set_code_extras,axis=0)

C:\Users\alcastan2201\AppData\Local\Temp\1\ipykernel_17416\2203617350.py:7: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df7=df6.loc[set_code_extras]


In [8]:
#Dictionaries
Index=df2.iloc[:,[0,1]].to_numpy()
dict_List_Conditions={'1-2 member households':lambda x: (x['Nr. of Active Individuals'] < 3),
                 '3-4 member households':lambda x: (x['Nr. of Active Individuals'] < 5) & ( x['Nr. of Active Individuals'] >= 3),
                 '5-6 member households':lambda x: (x['Nr. of Active Individuals'] < 7) & ( x['Nr. of Active Individuals'] >= 5),
                 '7+ member households':lambda x: x['Nr. of Active Individuals'] >= 7,
                 '5+ member households':lambda x: x['Nr. of Active Individuals'] >= 5,
                 'Traditional house': lambda x: x['Dwelling_Type'].str.contains('House'),
                 'Villa':lambda x: x['Dwelling_Type'].str.contains('Villa'),
                 'Apartment':lambda x: x['Dwelling_Type'].str.contains('Apartment'),
                 '1 TV set':lambda x: x['Nr. Monitored Tv'] ==1,
                 '2+ TV sets':lambda x: x['Nr. Monitored Tv'] >1}

bool_0=df1['Desinstallation_Date']>=Start
bool_1=(df1['Installation_Date'] <= Start) & (df1['Desinstallation_Date'].isnull())
bool_2=((df1['Installation_Date'] <= Start)&(bool_0))

bool_3=(df1['Status']=='199 -  Dropout')&(bool_0)
bool_4=(df1['Status']!='199 -  Dropout')

bool_5=(df1['Installation_Date']>=Start)

list_columns_validation_1=[('Total_Inst_Start_Date',(bool_1,bool_2)),('Dropout_Total',(bool_3,))]
list_columns_validation_2=[(coln,((bool_3)&(df1['Sec_Status']==coln),)) for coln in sorted(list_var_columns_99)]
list_columns_validation_3=[('Total_Newly_installed_in_Period',(bool_5,)),('Total_Installed_End_Date',(bool_4,))]
list_columns_validation_total=list_columns_validation_1+list_columns_validation_2+list_columns_validation_3

In [9]:
for tuple_columns in list_columns_validation_total:
    name_column,tuple_bool=tuple_columns
    list_data=[]
    print(name_column)
    for num_elem,tuple_elements in enumerate(Index):
        name_elem1,name_elem2=tuple_elements
        if num_elem < 11 : 
            if num_elem <2:
                namecol='Nationality'
            else:
                namecol='Region'
            bool_var=df1[namecol]==name_elem1

        else:
            bool_var=dict_List_Conditions[name_elem1](df1)
        if name_elem2 != 'Total':
            bool_var=((bool_var)&(df1['Nationality']==name_elem2))
        if len(tuple_bool)==2:    
            bool_First_Sum=((tuple_bool[0])&(bool_var))
            bool_Second_Sum=((tuple_bool[1])&(bool_var))
            First_Sum=bool_First_Sum.value_counts().get(True,0)
            Second_Sum=bool_Second_Sum.value_counts().get(True,0) 
            Total_Sum=First_Sum+Second_Sum
        else:
            bool_First_Sum=((tuple_bool[0])&(bool_var))
            Total_Sum=bool_First_Sum.value_counts().get(True,0)
        print('\t',tuple_elements,Total_Sum)
        list_data.append(Total_Sum)
    df2[name_column]=list_data

Total_Inst_Start_Date
	 ['Saudi' 'Total'] 1320
	 ['Non-Saudi' 'Total'] 930
	 ['Central Region' 'Total'] 700
	 ['Ad-Dammam' 'Total'] 108
	 ['Eastern Region -Other Cities' 'Total'] 193
	 ['Northern Region' 'Total'] 186
	 ['Southern Region' 'Total'] 140
	 ['Jeddah' 'Total'] 508
	 ['Madina' 'Total'] 187
	 ['Makkah Almukarramah' 'Total'] 123
	 ['Western Region -Other Cities' 'Total'] 105
	 ['1-2 member households' 'Saudi'] 158
	 ['3-4 member households' 'Saudi'] 395
	 ['5-6 member households' 'Saudi'] 471
	 ['7+ member households' 'Saudi'] 296
	 ['1-2 member households' 'Non-Saudi'] 185
	 ['3-4 member households' 'Non-Saudi'] 312
	 ['5+ member households' 'Non-Saudi'] 433
	 ['Traditional house' 'Total'] 269
	 ['Villa' 'Total'] 539
	 ['Apartment' 'Total'] 1442
	 ['1 TV set' 'Total'] 1477
	 ['2+ TV sets' 'Total'] 773
Dropout_Total
	 ['Saudi' 'Total'] 45
	 ['Non-Saudi' 'Total'] 25
	 ['Central Region' 'Total'] 23
	 ['Ad-Dammam' 'Total'] 9
	 ['Eastern Region -Other Cities' 'Total'] 5
	 ['Norther

	 ['3-4 member households' 'Saudi'] 5
	 ['5-6 member households' 'Saudi'] 5
	 ['7+ member households' 'Saudi'] 3
	 ['1-2 member households' 'Non-Saudi'] 1
	 ['3-4 member households' 'Non-Saudi'] 3
	 ['5+ member households' 'Non-Saudi'] 3
	 ['Traditional house' 'Total'] 3
	 ['Villa' 'Total'] 6
	 ['Apartment' 'Total'] 14
	 ['1 TV set' 'Total'] 18
	 ['2+ TV sets' 'Total'] 5
99U - Forced Turnover - Untrustworthy home
	 ['Saudi' 'Total'] 1
	 ['Non-Saudi' 'Total'] 0
	 ['Central Region' 'Total'] 1
	 ['Ad-Dammam' 'Total'] 0
	 ['Eastern Region -Other Cities' 'Total'] 0
	 ['Northern Region' 'Total'] 0
	 ['Southern Region' 'Total'] 0
	 ['Jeddah' 'Total'] 0
	 ['Madina' 'Total'] 0
	 ['Makkah Almukarramah' 'Total'] 0
	 ['Western Region -Other Cities' 'Total'] 0
	 ['1-2 member households' 'Saudi'] 0
	 ['3-4 member households' 'Saudi'] 0
	 ['5-6 member households' 'Saudi'] 1
	 ['7+ member households' 'Saudi'] 0
	 ['1-2 member households' 'Non-Saudi'] 0
	 ['3-4 member households' 'Non-Saudi'] 0
	 ['5+ 

In [10]:
validationcolumn=df2[list_var_columns_99].sum(axis=1)
df2.insert(5,'Validation',validationcolumn)
Comparation=(df2['Dropout_Total']==df2['Validation'])
#Comparation.value_counts()

In [11]:
#Validación manual
Nacionalidades=df2['Total_Inst_Start_Date'][0:2].sum()
Regiones=df2['Total_Inst_Start_Date'][2:11].sum()
members_saudi=df2['Total_Inst_Start_Date'][11:15].sum()
members_nonsaudi=df2['Total_Inst_Start_Date'][15:18].sum()
members=members_nonsaudi+members_saudi
members

2250

In [12]:
tv=df2['Total_Inst_Start_Date'][21:23].sum()
DwellingType=df2['Total_Inst_Start_Date'][18:21].sum()
Members=df2['Total_Inst_Start_Date'][21:23].sum()

In [13]:
#df4
df1['Total_Panel_Months'] = abs((df1['Desinstallation_Date']-df1['Installation_Date'])/np.timedelta64(1, 'M'))
df1['Total_Panel_Months'] = df1['Total_Panel_Months'].astype(dtype=int,errors = 'ignore')
panel1=df1[df1['Desinstallation_Date']>=Start]
panel2=panel1[panel1['Sec_Status'].str.startswith('99')]
listdate=panel2[['Sec_Status','Installation_Date','Desinstallation_Date','Total_Panel_Months']]

In [18]:
dict_df1_Conditions={'0_Months':lambda x: x['Total_Panel_Months'] < 1,
'1-3_Months':lambda x: (x['Total_Panel_Months'] >= 1) &( x['Total_Panel_Months'] < 4),
'4-6_Months':lambda x: (x['Total_Panel_Months'] >= 4) &( x['Total_Panel_Months'] < 7),
'7-12_Months':lambda x: (x['Total_Panel_Months']>= 7) &( x['Total_Panel_Months'] < 13)}


matrix=[]
list_keys=dict_df1_Conditions.keys()

for i in list_var_columns_99:
    list_temp=[]
    for j in list_keys:
        value=dict_df1_Conditions[j](listdate\
                                                [listdate['Sec_Status']==i])\
        .value_counts(normalize=False).get(True,0)
        list_temp.append(value)
    matrix.append(list_temp)

df4=pd.DataFrame(matrix)
df4.columns = ['0_Months','1-3_Months', '4-6_Months', '7-12_Months']
df4.index=list_var_columns_99
df4['Total']=df4.sum(axis=1)
df4.loc["Total"] = df4.sum(axis=0)
df4.loc["% Over Current Installed"] = round((df4.sum(axis=0)/df2['Total_Inst_Start_Date'][21:23].sum())*100,2)
df4.loc["% Over Current Deinstalled"] = round((df4.loc["Total"]/(float(df4.loc["Total"][4])))*100,2)
df4

,0_Months,1-3_Months,4-6_Months,7-12_Months,Total
99T - Forced Turnover - Non-Cooperating home,1.00,2.00,4.00,16.00,23.00
99X - Privacy concern / Concerned about Privacy,0.00,0.00,1.00,2.00,3.00
99U - Forced Turnover - Untrustworthy home,0.00,0.00,0.00,1.00,1.00
99K - Forced Turnover - QC No Compliance,1.00,1.00,2.00,3.00,7.00
99N - Forced Turnover - Refuse to have all TV installed,1.00,1.00,1.00,1.00,4.00
99D - PM request - Family reason,2.00,3.00,1.00,7.00,13.00
99C - Forced Turnover - PM Moving,0.00,1.00,3.00,1.00,5.00
99H - PM request - Heavy electricity expenses,0.00,0.00,0.00,0.00,0.00
99M - Forced Turnover - Technical,0.00,0.00,1.00,0.00,1.00
99E - PM request - Too troublesome,0.00,1.00,1.00,1.00,3.00


In [15]:
filename='dropout_output.xlsx'
with pd.ExcelWriter(filename) as writer:  # doctest: +SKIP
    df1.to_excel(writer, sheet_name='Base')
    df2.to_excel(writer, sheet_name='Report')
    df3.to_excel(writer, sheet_name='Region Mapping')
    df4.to_excel(writer, sheet_name='Panel Distribution Age')
    df7.to_excel(writer, sheet_name='Extras Validation')